## GETTING STATS FOR NORMALIZATION

In [3]:
from PIL import Image
import rasterio as rio
from rasterio.plot import show
import numpy as np
import os
import glob
import random
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import tifffile
from scipy.ndimage import sobel
from skimage import color, io
import cv2
from scipy.ndimage import gaussian_filter
import time
import pandas as pd
import warnings
from tqdm import tqdm

## Trial with subset of TIF images

In [13]:
# Selecting all the files in the folder
features_folder = '../Data/train_features.tar_MLIC14m/train_satellite/'
all_files = list(glob.glob(os.path.join(features_folder, '*')))


print(len(all_files))
print(all_files)

# Extracting image IDs
id_list = []

for file_path in all_files:
    start_index = file_path.rfind('\\') + 1
    end_index = file_path.rfind('_')
    result = file_path[start_index:end_index]
    id_list.append(result)

print(id_list)

# Import CSV file
metadata = '20240129_filter_metadata_formodel.csv'  # Replace with your CSV file path
df = pd.read_csv(metadata)

# Filter id_list based on the CSV
id_list = [img_id for img_id in id_list if img_id in df['image_id'].values]

print(len(id_list))


def stats_norm_band(band_flat, band_dict, clouds_flat):
    count_per_val = {}

    # Get the indices where mask is 0
    valid_indices = np.where(clouds_flat == 0)

    # Get the pixel values for the valid indices
    valid_px_val = band_flat[valid_indices]

    # Update the dictionary for the current band
    for pixel_value in valid_px_val:
        band_dict[pixel_value] = band_dict.get(pixel_value, 0) + 1

    return band_dict


swir_dict = {}
nir_dict = {}
red_dict = {}
green_dict = {}
blue_dict = {}

pbar = tqdm(total=len(id_list), desc="Processing Images", unit="image")

for img_id in id_list:

    features_img =  os.path.join(features_folder, f'{img_id}_satellite.tif')

    with rio.open(features_img) as src:
        swir = src.read(1)
        nir = src.read(2)
        red = src.read(3)
        green = src.read(4)
        blue = src.read(5)
        clouds = src.read(6)

    clouds_flat = clouds.flatten()
    swir_flat = swir.flatten()
    nir_flat = nir.flatten()
    red_flat =  red.flatten()
    green_flat = green.flatten()
    blue_flat = blue.flatten()

    swir_dict = stats_norm_band(swir_flat, swir_dict, clouds_flat)
    nir_dict = stats_norm_band(nir_flat, nir_dict, clouds_flat)
    red_dict = stats_norm_band(red_flat, red_dict, clouds_flat)
    green_dict = stats_norm_band(green_flat, green_dict, clouds_flat)
    blue_dict = stats_norm_band(blue_flat, blue_dict, clouds_flat)

        # Increment the tqdm progress bar
    pbar.update(1)
    time.sleep(0.1)  # Simulate some work being done

5637
['../Data/train_features.tar_MLIC14m/train_satellite\\AA498489_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AA648736_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AA800151_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AA878727_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB160377_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB417661_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB440131_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB602248_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB678991_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB795781_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB841632_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB970708_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AC147

Processing Images:   0%|          | 0/5635 [02:14<?, ?image/s]
c:\Users\cande\anaconda3\envs\spatialenv\lib\site-packages\rasterio\__init__.py:333: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


AttributeError: 'NpzFile' object has no attribute 'where'

In [6]:
nir_dist_df = pd.DataFrame(list(nir_dict.items()), columns=['px_val', 'num_pxs'])
nir_dist_df.to_csv('../KELP_Mapping/bands_distribution/nir_distribution.csv', index=False)

swir_dist_df = pd.DataFrame(list(swir_dict.items()), columns=['px_val', 'num_pxs'])
swir_dist_df.to_csv('../KELP_Mapping/bands_distribution/swir_distribution.csv', index=False)

red_dist_df = pd.DataFrame(list(red_dict.items()), columns=['px_val', 'num_pxs'])
red_dist_df.to_csv('../KELP_Mapping/bands_distribution/red_distribution.csv', index=False)

green_dist_df = pd.DataFrame(list(green_dict.items()), columns=['px_val', 'num_pxs'])
green_dist_df.to_csv('../KELP_Mapping/bands_distribution/green_distribution.csv', index=False)

blue_dist_df = pd.DataFrame(list(blue_dict.items()), columns=['px_val', 'num_pxs'])
blue_dist_df.to_csv('../KELP_Mapping/bands_distribution/blue_distribution.csv', index=False)

In [49]:
print(nir_dict)

{7865: 599, 8351: 44, 8108: 74, 7864: 340, 7619: 37824, 7620: 44711, 7863: 7, 8107: 29, 8837: 7, 8594: 9, 56199: 1, 65535: 46, 49326: 1, 5170: 2, 6644: 3, 8109: 274, 8352: 68, 7377: 53, 7866: 549, 7621: 7415, 9321: 3, 8836: 6, 8595: 33, 7622: 712, 8110: 96, 7867: 350, 8353: 87, 8596: 59, 7623: 5794, 9323: 15, 9080: 225, 7868: 200, 8838: 15, 9806: 9, 8354: 77, 8597: 38, 7380: 110, 9322: 7, 9563: 6, 9324: 3, 9565: 5, 9564: 12, 9805: 12, 10288: 7, 10047: 10, 8111: 54, 7378: 80, 7624: 8301, 9081: 127, 7379: 72, 7869: 116, 8355: 35, 7381: 110, 8839: 16, 9082: 17, 8112: 45, 7625: 1521, 8113: 40, 8598: 15, 8840: 5, 7382: 170, 7870: 65, 8356: 11, 7626: 3621, 5177: 4, 7383: 196, 42604: 1, 43223: 1, 24714: 1, 11965: 1, 5423: 1, 13631: 6, 27618: 1, 44043: 1, 48504: 1, 5178: 2, 7138: 245, 27396: 1, 36739: 1, 53063: 1, 7627: 1077, 7384: 220, 7628: 1053, 7871: 130, 8114: 43, 7385: 220, 8599: 24, 7872: 174, 8357: 27, 8115: 97, 7629: 917, 8600: 28, 8358: 41, 7386: 363, 7873: 48, 7630: 769, 8116: 127, 

## GET DEM DISTRIBUTION

In [9]:
# Selecting all the files in the folder
features_folder = '../Data/train_features.tar_MLIC14m/train_satellite/'
all_files = list(glob.glob(os.path.join(features_folder, '*')))


print(len(all_files))
print(all_files)

# Extracting image IDs
id_list = []

for file_path in all_files:
    start_index = file_path.rfind('\\') + 1
    end_index = file_path.rfind('_')
    result = file_path[start_index:end_index]
    id_list.append(result)

print(id_list)

# Import CSV file
metadata = '20240129_filter_metadata_formodel.csv'  # Replace with your CSV file path
df = pd.read_csv(metadata)

# Filter id_list based on the CSV
id_list = [img_id for img_id in id_list if img_id in df['image_id'].values]

print(len(id_list))


def stats_norm_band(band_flat, band_dict):
    count_per_val = {}

    # Update the dictionary for the current band
    for pixel_value in band_flat:
        band_dict[pixel_value] = band_dict.get(pixel_value, 0) + 1

    return band_dict


dist_dict = {}

pbar = tqdm(total=len(id_list), desc="Processing Images", unit="image")

for img_id in id_list:

    features_img =  os.path.join(features_folder, f'{img_id}_satellite.tif')

    with rio.open(features_img) as src:
        dist = src.read(6)

    dist_flat = dist.flatten()

    dist_dict = stats_norm_band(dist_flat, dist_dict)

        # Increment the tqdm progress bar
    pbar.update(1)
    time.sleep(0.00001)  # Simulate some work being done

5637
['../Data/train_features.tar_MLIC14m/train_satellite\\AA498489_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AA648736_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AA800151_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AA878727_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB160377_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB417661_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB440131_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB602248_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB678991_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB795781_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB841632_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AB970708_satellite.tif', '../Data/train_features.tar_MLIC14m/train_satellite\\AC147

Processing Images:   0%|          | 0/3183 [00:00<?, ?image/s]c:\Users\cande\anaconda3\envs\spatialenv\lib\site-packages\rasterio\__init__.py:333: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
Processing Images: 100%|██████████| 3183/3183 [02:31<00:00, 21.31image/s]

In [10]:
dem_dist_df = pd.DataFrame(list(dem_dict.items()), columns=['px_val', 'num_pxs'])
dem_dist_df.to_csv('../KELP_Mapping/bands_distribution/dem_distribution.csv', index=False)

# GET DIST DISTR

In [5]:
# Selecting all the files in the folder
features_folder = '../Data/train_satellite_np'
all_files = list(glob.glob(os.path.join(features_folder, '*')))


print(len(all_files))
print(all_files)

# Extracting image IDs
id_list = []

for file_path in all_files:
    start_index = file_path.rfind('\\') + 1
    end_index = file_path.rfind('_')
    result = file_path[start_index:end_index]
    id_list.append(result)

print(id_list)

# Import CSV file
metadata = '20240129_filter_metadata_formodel.csv'  # Replace with your CSV file path
df = pd.read_csv(metadata)

# Filter id_list based on the CSV
id_list = [img_id for img_id in id_list if img_id in df['image_id'].values]

print(len(id_list))


def stats_norm_band(band_flat, band_dict):
    count_per_val = {}

    # Update the dictionary for the current band
    for pixel_value in band_flat:
        band_dict[pixel_value] = band_dict.get(pixel_value, 0) + 1

    return band_dict


dist_dict = {}

pbar = tqdm(total=len(id_list), desc="Processing Images", unit="image")

for img_id in id_list:

    features_img =  os.path.join(features_folder, f'{img_id}_satellite.npz')

    dist = np.load(features_img)
    dist_arr = dist['arr_0']
    dist_map = dist_arr[7,:,:]
    dist_map = np.round(dist_map).astype(int)

    dist_flat = dist_map.flatten()

    dist_dict = stats_norm_band(dist_flat, dist_dict)

        # Increment the tqdm progress bar
    pbar.update(1)
    time.sleep(0.00001)  # Simulate some work being done

5635
['../Data/train_satellite_np\\AA498489_satellite.npz', '../Data/train_satellite_np\\AA648736_satellite.npz', '../Data/train_satellite_np\\AA800151_satellite.npz', '../Data/train_satellite_np\\AA878727_satellite.npz', '../Data/train_satellite_np\\AB160377_satellite.npz', '../Data/train_satellite_np\\AB417661_satellite.npz', '../Data/train_satellite_np\\AB440131_satellite.npz', '../Data/train_satellite_np\\AB602248_satellite.npz', '../Data/train_satellite_np\\AB678991_satellite.npz', '../Data/train_satellite_np\\AB795781_satellite.npz', '../Data/train_satellite_np\\AB841632_satellite.npz', '../Data/train_satellite_np\\AB970708_satellite.npz', '../Data/train_satellite_np\\AC147670_satellite.npz', '../Data/train_satellite_np\\AC582589_satellite.npz', '../Data/train_satellite_np\\AC664566_satellite.npz', '../Data/train_satellite_np\\AC930244_satellite.npz', '../Data/train_satellite_np\\AD125151_satellite.npz', '../Data/train_satellite_np\\AD154970_satellite.npz', '../Data/train_satelli

Processing Images: 100%|█████████▉| 3181/3182 [05:03<00:00,  9.98image/s]

Processing Images: 100%|██████████| 3182/3182 [05:20<00:00,  9.98image/s]

In [6]:
dist_dist_df = pd.DataFrame(list(dist_dict.items()), columns=['px_val', 'num_pxs'])
dist_dist_df.to_csv('../KELP_Mapping/bands_distribution/dist_distribution.csv', index=False)

In [5]:
def stats_norm_band(band_flat, band_dict):
    count_per_val = {}

    # Update the dictionary for the current band
    for pixel_value in band_flat:
        band_dict[pixel_value] = band_dict.get(pixel_value, 0) + 1

    return band_dict

## STATS FOR NDVI


In [13]:
# Selecting all the files in the folder
features_folder = '../Data/NDVI_train_np'
all_files = list(glob.glob(os.path.join(features_folder, '*')))

print(len(all_files))
print(all_files)

# Extracting image IDs
id_list = []

for file_path in all_files:
    start_index = file_path.rfind('\\') + 1
    end_index = file_path.rfind('_')
    result = file_path[start_index:end_index]
    id_list.append(result)

print(id_list)

# Import CSV file
metadata = '20240129_filter_metadata_formodel.csv'  # Replace with your CSV file path
df = pd.read_csv(metadata)

# Filter id_list based on the CSV
id_list = [img_id for img_id in id_list if img_id in df['image_id'].values]

print(len(id_list))

dist_dict = {}

pbar = tqdm(total=len(id_list), desc="Processing Images", unit="image")

for img_id in id_list:

    features_img =  os.path.join(features_folder, f'{img_id}_satellite.npz')

    dist = np.load(features_img)
    dist_arr = dist['arr_0']

    dist_arr = dist_arr*1000

    dist_map = np.round(dist_arr).astype(int)

    dist_flat = dist_map.flatten()

    dist_dict = stats_norm_band(dist_flat, dist_dict)

        # Increment the tqdm progress bar
    pbar.update(1)
    time.sleep(0.00001)  # Simulate some work being done

5635
['../Data/NDVI_train_np\\AA498489_satellite.npz', '../Data/NDVI_train_np\\AA648736_satellite.npz', '../Data/NDVI_train_np\\AA800151_satellite.npz', '../Data/NDVI_train_np\\AA878727_satellite.npz', '../Data/NDVI_train_np\\AB160377_satellite.npz', '../Data/NDVI_train_np\\AB417661_satellite.npz', '../Data/NDVI_train_np\\AB440131_satellite.npz', '../Data/NDVI_train_np\\AB602248_satellite.npz', '../Data/NDVI_train_np\\AB678991_satellite.npz', '../Data/NDVI_train_np\\AB795781_satellite.npz', '../Data/NDVI_train_np\\AB841632_satellite.npz', '../Data/NDVI_train_np\\AB970708_satellite.npz', '../Data/NDVI_train_np\\AC147670_satellite.npz', '../Data/NDVI_train_np\\AC582589_satellite.npz', '../Data/NDVI_train_np\\AC664566_satellite.npz', '../Data/NDVI_train_np\\AC930244_satellite.npz', '../Data/NDVI_train_np\\AD125151_satellite.npz', '../Data/NDVI_train_np\\AD154970_satellite.npz', '../Data/NDVI_train_np\\AD274174_satellite.npz', '../Data/NDVI_train_np\\AD480612_satellite.npz', '../Data/NDVI_

Processing Images: 0image [01:14, ?image/s][00:00<?, ?image/s]
Processing Images: 100%|█████████▉| 3181/3182 [01:47<00:00, 29.36image/s]

In [14]:
ndvi_dist_df = pd.DataFrame(list(dist_dict.items()), columns=['px_val', 'num_pxs'])
ndvi_dist_df.to_csv('Normalization/bands_distribution/ndvi_distribution.csv', index=False)

### MNDWI STATS

In [16]:
# Selecting all the files in the folder
features_folder = '../Data/MNDWI_train_np'
all_files = list(glob.glob(os.path.join(features_folder, '*')))

print(len(all_files))
print(all_files)

# Extracting image IDs
id_list = []

for file_path in all_files:
    start_index = file_path.rfind('\\') + 1
    end_index = file_path.rfind('_')
    result = file_path[start_index:end_index]
    id_list.append(result)

# Import CSV file
metadata = '20240129_filter_metadata_formodel.csv'  # Replace with your CSV file path
df = pd.read_csv(metadata)

# Filter id_list based on the CSV
id_list = [img_id for img_id in id_list if img_id in df['image_id'].values]

print(len(id_list))

dist_dict = {}

pbar = tqdm(total=len(id_list), desc="Processing Images", unit="image")

for img_id in id_list:

    features_img =  os.path.join(features_folder, f'{img_id}_satellite.npz')

    dist = np.load(features_img)
    dist_arr = dist['arr_0']

    dist_arr = dist_arr*1000

    dist_map = np.round(dist_arr).astype(int)

    dist_flat = dist_map.flatten()

    dist_dict = stats_norm_band(dist_flat, dist_dict)

        # Increment the tqdm progress bar
    pbar.update(1)
    time.sleep(0.00001)  # Simulate some work being done

5635
['../Data/MNDWI_train_np\\AA498489_satellite.npz', '../Data/MNDWI_train_np\\AA648736_satellite.npz', '../Data/MNDWI_train_np\\AA800151_satellite.npz', '../Data/MNDWI_train_np\\AA878727_satellite.npz', '../Data/MNDWI_train_np\\AB160377_satellite.npz', '../Data/MNDWI_train_np\\AB417661_satellite.npz', '../Data/MNDWI_train_np\\AB440131_satellite.npz', '../Data/MNDWI_train_np\\AB602248_satellite.npz', '../Data/MNDWI_train_np\\AB678991_satellite.npz', '../Data/MNDWI_train_np\\AB795781_satellite.npz', '../Data/MNDWI_train_np\\AB841632_satellite.npz', '../Data/MNDWI_train_np\\AB970708_satellite.npz', '../Data/MNDWI_train_np\\AC147670_satellite.npz', '../Data/MNDWI_train_np\\AC582589_satellite.npz', '../Data/MNDWI_train_np\\AC664566_satellite.npz', '../Data/MNDWI_train_np\\AC930244_satellite.npz', '../Data/MNDWI_train_np\\AD125151_satellite.npz', '../Data/MNDWI_train_np\\AD154970_satellite.npz', '../Data/MNDWI_train_np\\AD274174_satellite.npz', '../Data/MNDWI_train_np\\AD480612_satellite.

Processing Images: 100%|█████████▉| 3181/3182 [03:35<00:00, 14.94image/s]

In [17]:
mndwi_dist_df = pd.DataFrame(list(dist_dict.items()), columns=['px_val', 'num_pxs'])
mndwi_dist_df.to_csv('Normalization/bands_distribution/mndwi_distribution.csv', index=False)